In [1]:
# Import tutto quanto
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy

In [ ]:
# LOADING DATA

df=pd.read_csv("ICU_Challenge_Dataset.csv", sep=",")
df_description = df.describe()
columns = df.columns


In [ ]:
# MISSING DATA - da esplorare
# print(df[df.columns[df.isna().any()]])
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
print(missing_value_df)

In [ ]:
df_col_keep = missing_value_df[missing_value_df['percent_missing']<50]
df_col = df_col_keep['column_name']

df=df[df_col]
df

In [ ]:
print(np.unique(df.dtypes))

int_var = list(df.columns[df.dtypes == np.int64])
float_var = list(df.columns[df.dtypes == np.float64])

cat_var = []
num_var = []
for cat in list(df.columns):
    if len(df[cat].unique()) <=3: #conta anche i Nan come valori
        cat_var.append(cat)
    else:
        num_var.append(cat)

categorical_df = df[cat_var]
numerical_df = df[num_var]

In [ ]:
#normal test and for similarity
#different
df = df.drop(['BUN_first', 'Creatinine_first', 'Na_first', 'Weight_first'], axis=1)
df_copia = df
df

In [ ]:
df['HCT_diff'] = df['HCT_first'] - df['HCT_last']
df['HR_diff'] = df['HR_first'] - df['HR_last']
df['Temp_diff'] = df['Temp_first'] - df['Temp_last']
df['GCS_diff'] = df['GCS_first'] - df['GCS_last']
df['Platelets_diff'] = df['Platelets_first'] - df['Platelets_last']
df['HCO3_diff'] = df['HCO3_first'] - df['HCO3_last']
df['K_diff'] = df['K_first'] - df['K_last']
df['WBC_diff'] = df['WBC_first'] - df['WBC_last']
df['Mg_diff'] = df['Mg_first'] - df['Mg_last']
df['Glucose_diff'] = df['Glucose_first'] - df['Glucose_last']
df['NISysABP_diff'] = df['NISysABP_first'] - df['NISysABP_last']
df['NIDiasABP_diff'] = df['NIDiasABP_first'] - df['NIDiasABP_last']
df['pH_diff'] = df['pH_first'] - df['pH_last']
df['PaO2_diff'] = df['PaO2_first'] - df['PaO2_last']
df['PaCO2_diff'] = df['PaCO2_first'] - df['PaCO2_last']
df['MAP_diff'] = df['MAP_first'] - df['MAP_last']
df['SysABP_diff'] = df['SysABP_first'] - df['SysABP_last']
df['DiasABP_diff'] = df['DiasABP_first'] - df['DiasABP_last']
df['FiO2_diff'] = df['FiO2_first'] - df['FiO2_last']
df['Lactate_diff'] = df['Lactate_first'] - df['Lactate_last']

df = df.drop(['HCT_first', 'HCT_last', 'HR_first', 'HR_last', 'Temp_first', 'Temp_last', 'GCS_first', 'GCS_last', 'Platelets_first', 'Platelets_last', 'HCO3_first', 'HCO3_last', 'K_first', 'K_last', 'WBC_first', 'WBC_last', 'Mg_first', 'Mg_last', 'Glucose_first', 'Glucose_last', 'NISysABP_first', 'NISysABP_last', 'NIDiasABP_first', 'NIDiasABP_last', 'pH_first', 'pH_last', 'PaO2_first', 'PaO2_last', 'PaCO2_first', 'PaCO2_last',  'MAP_first', 'MAP_last', 'SysABP_first', 'SysABP_last', 'DiasABP_first', 'DiasABP_last', 'FiO2_first', 'FiO2_last', 'Lactate_first', 'Lactate_last' ], axis=1)
df

In [ ]:
def plot_correlation_heatmap(df):
    plt.figure(figsize=(20,20))
    corr = df.corr()
    ax = sns.heatmap(
        corr,
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True, annot=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    )
    ax.set_yticklabels(
        ax.get_yticklabels(),
        rotation=45,

    );

In [ ]:
cat_var = []
num_var = []
for cat in list(df.columns):
    if len(df[cat].unique()) <=3: #conta anche i Nan come valori
        cat_var.append(cat)
    else:
        num_var.append(cat)

categorical_df = df[cat_var]
numerical_df = df[num_var]

In [ ]:
numerical_df

In [ ]:
numerical_df
to_drop = []

for x in list(numerical_df.columns):
    if 'highest' in x:
        to_drop.append(x)
    if 'median' in x:
        to_drop.append(x)
    if 'lowest' in x:
        to_drop.append(x)
to_drop

In [ ]:
numerical_df = numerical_df.drop(to_drop, axis=1)
numerical_df

In [ ]:
numerical_df
plot_correlation_heatmap(numerical_df)

In [ ]:
mean_values = numerical_df.mean()
mean_values

In [ ]:
numerical_df = numerical_df.fillna(value=mean_values)
numerical_df.isnull().sum()

In [ ]:
numerical_df.hist(figsize=(10,10))

In [ ]:
import matplotlib.pyplot as plt
crea_log = np.log(numerical_df['Creatinine_last'])

In [ ]:
import matplotlib.pyplot as plt
bun_log= np.log(numerical_df['BUN_last'])
sns.distplot(bun_log)
fig = plt.figure()
bun_log.hist()

In [ ]:
numerical_df=numerical_df.drop(['Creatinine_last', 'BUN_last'], axis=1)
numerical_df.insert(2, 'Creatinine_last_log', crea_log)
numerical_df.insert(3, 'BUN_last_log', bun_log)
numerical_df

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(numerical_df)
scaled_df=pd.DataFrame(scaler.transform(numerical_df))
scaled_df.columns = numerical_df.columns

scaled_df

In [ ]:
frames=[scaled_df, categorical_df]
df_3 = pd.concat(frames, axis=1)
df_3

In [ ]:
categorical_df.isnull().sum()

In [ ]:
#tolgo righe con genere nan
indexNames = df_3[df_3['Gender'].isnull()].index
df_3.drop(indexNames, inplace=True)
df_3

In [ ]:
#tolgo righe con mech nan
indexNames = df_3[df_3['MechVentLast8Hour'].isnull()].index
df_3.drop(indexNames, inplace=True)
df_3

In [ ]:
plot_correlation_heatmap(df_3)

In [ ]:
df_3=df_3.drop(['recordid'], axis=1)
df_3

In [ ]:
df_3.to_csv('df_3.csv', index=False)